# Current To-Do:
- Contrast map comparison
- Conjunction analysis

Backburner questions:
- XCP-D?
- DiFuMo atlas instead of Schaefer?
- Different SRM distance penalties (distance as penalty instead of parcelwise? Searchlights instead of parcels?)

[ROADMAP DOC](https://docs.google.com/document/d/13P4QTHxrT5lZfCOXtN59xCKpJfnObtqh3uZkuRqPxR4/edit?pli=1#heading=h.2qncjqtc0b5j)

# Testing

In [ ]:
import nibabel as nib
PATH = '/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS_21.0.1/derivatives/output_optcom_MNI/flanker_lev1_output/task_flanker_rtmodel_rt_centered/contrast_estimates/'
testfile = 'sub-s43_task-flanker_contrast-incongruent - congruent_rtmodel-rt_centered_stat-fixed-effects_t-test.nii.gz'
test = nib.load(PATH + testfile)

testfile2 = 'sub-s19_task-flanker_contrast-incongruent - congruent_rtmodel-rt_centered_stat-fixed-effects_t-test.nii.gz'
test2 = nib.load(PATH + testfile2)


import itertools
import numpy as np
output = {}
for task in ['flanker','spatialTS','cuedTS','directedForgetting','stopSignal','goNogo', 'shapeMatching', 'nBack']:
    output[task] = {
        'srm': {'all_overlap': [], 'all_dice': []},
        'nosrm': {'all_overlap': [], 'all_dice': []}
    }
    subjects = ['sub-s01', 'sub-s02', 'sub-s03', 'sub-s04', 'sub-s05']

    for sub1, sub2 in itertools.combinations(subjects, 2):
        output[task]['nosrm']['all_overlap'].append(np.random.randint(0,10,5))
        output[task]['nosrm']['all_dice'].append(np.random.randint(0,10,5))
        output[task]['srm']['all_overlap'].append(np.random.randint(0,10,5))
        output[task]['srm']['all_dice'].append(np.random.randint(0,10,5))

    for method in ['srm', 'nosrm']:
        output[task][method]['avg_overlap'] = np.mean(output[task][method]['all_overlap'])
        output[task][method]['avg_dice'] = np.mean(output[task][method]['all_dice'])